# SparkDataFrames

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pandas

## Imports & Load Data

In [ ]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

import findspark

findspark.init()

df = pd.read_csv("cleaned_music_streaming.csv")

df.head()

## Create Spark Context and Sessoin

In [ ]:
# Context
SparkContext = SparkContext()

# Session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark_df = spark.createDataFrame(df)

## PrintSchema & Show

In [ ]:
spark_df.printSchema()

In [ ]:
spark_df.show(5)

## Temp View of Data

In [ ]:
spark_df.createOrReplaceTempView("music_streaming")